<a href="https://colab.research.google.com/github/Srikara2005/Data-Analytics-Lab/blob/main/Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Lab-1 Data preprocessing**

Data preprocessing is the process of transforming raw, often messy data into a clean and understandable format that is suitable for analysis or machine learning models.

Real-world data is often incomplete, inconsistent, and lacking in certain behaviors or trends, and is likely to contain many errors.

Handling Missing Values: Filling in gaps (imputation) or removing incomplete rows.

Noisy Data: smoothing out error and outliers (binning, regression, clustering).

Outlier Removal: Identifying data points that are statistically improbable (e.g., Age = 200).

Identifying redundancy involves finding duplicate records or attributes that convey the same information (e.g., storing both "Age" and "Date of Birth").

Elimination removes these repetitive instances to reduce dataset size, ensure consistency, and prevent the model from becoming biased toward frequent data points.

In [ ]:
import pandas as pd
import numpy as np

**Create a Dataset**


In [ ]:
data = {
    'Name': ['Akshay', 'Mahesh Bob', 'Akshay', 'Kalyan babu', 'Bhaai', 'PraBoss', 'Young Tiger', 'Cherry'],
    'Age': [25, np.nan, 25, 45, 120, 30, np.nan, 35],  # 120 is likely noise/outlier
    'Salary': [50000, 60000, 50000, 80000, 55000, 58000, 62000, 2000000], # 2M is noise
    'City': ['Nizambad', 'Lucknow', 'Nizambad', 'Chilka nagar', 'Hyderabad', 'Pittapuram', 'Nizambad', 'Saroor nagar']
}

df = pd.DataFrame(data)

print("--- ORIGINAL DATAFRAME ---")
print(df)
print("\n")

--- ORIGINAL DATAFRAME ---
          Name    Age   Salary          City
0       Akshay   25.0    50000      Nizambad
1   Mahesh Bob    NaN    60000       Lucknow
2       Akshay   25.0    50000      Nizambad
3  Kalyan babu   45.0    80000  Chilka nagar
4        Bhaai  120.0    55000     Hyderabad
5      PraBoss   30.0    58000    Pittapuram
6  Young Tiger    NaN    62000      Nizambad
7       Cherry   35.0  2000000  Saroor nagar




**Handling missing values**





In [ ]:
df.describe()


,Age,Salary
count,6.000000,8.000000e+00
mean,46.666667,3.018750e+05
std,36.696957,6.862116e+05
min,25.000000,5.000000e+04
25%,26.250000,5.375000e+04
50%,32.500000,5.900000e+04
75%,42.500000,6.650000e+04
max,120.000000,2.000000e+06


In [ ]:
# Check for missing values
print(f"Missing values per column:\n{df.isnull().sum()}\n")

Missing values per column:
Name      0
Age       2
Salary    0
City      0
dtype: int64



In [ ]:
# Method 1: Drop rows with missing values (Destructive)
df_dropped = df.dropna()
print("1. Shape after dropping rows with NaNs:", df_dropped.shape)

print(df_dropped)

1. Shape after dropping rows with NaNs: (6, 4)
          Name    Age   Salary          City
0       Akshay   25.0    50000      Nizambad
2       Akshay   25.0    50000      Nizambad
3  Kalyan babu   45.0    80000  Chilka nagar
4        Bhaai  120.0    55000     Hyderabad
5      PraBoss   30.0    58000    Pittapuram
7       Cherry   35.0  2000000  Saroor nagar


In [ ]:
# Method 2: Imputation (Filling with Mean/Median/Mode)
# We will use Median for Age to fill NaNs (robust to outliers)
df_imputed = df.copy()
median_age = df_imputed['Age'].median()
df_imputed['Age'] = df_imputed['Age'].fillna(median_age)

print(f"2. Filled missing Age with median ({median_age}):")
print(df_imputed)
print("\n")

2. Filled missing Age with median (32.5):
          Name    Age   Salary          City
0       Akshay   25.0    50000      Nizambad
1   Mahesh Bob   32.5    60000       Lucknow
2       Akshay   25.0    50000      Nizambad
3  Kalyan babu   45.0    80000  Chilka nagar
4        Bhaai  120.0    55000     Hyderabad
5      PraBoss   30.0    58000    Pittapuram
6  Young Tiger   32.5    62000      Nizambad
7       Cherry   35.0  2000000  Saroor nagar




**NOISE DETECTION & REMOVAL (Outliers)**

In [ ]:
# We will use the IQR (Interquartile Range) method to remove Age outliers (e.g., 120).

Q1 = df_imputed['Age'].quantile(0.25)
Q3 = df_imputed['Age'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Age Bounds: {lower_bound} to {upper_bound}")

# Filter the data
df_clean_noise = df_imputed[
    (df_imputed['Age'] >= lower_bound) &
    (df_imputed['Age'] <= upper_bound)
]

print("Rows removed (Noise):")
print(df_imputed[~df_imputed.index.isin(df_clean_noise.index)])
print("\n")
print("Dataset after noise removal")
print(df_clean_noise)

Age Bounds: 15.625 to 50.625
Rows removed (Noise):
    Name    Age  Salary       City
4  Bhaai  120.0   55000  Hyderabad


Dataset after noise removal
          Name   Age   Salary          City
0       Akshay  25.0    50000      Nizambad
1   Mahesh Bob  32.5    60000       Lucknow
2       Akshay  25.0    50000      Nizambad
3  Kalyan babu  45.0    80000  Chilka nagar
5      PraBoss  30.0    58000    Pittapuram
6  Young Tiger  32.5    62000      Nizambad
7       Cherry  35.0  2000000  Saroor nagar


**IDENTIFYING & ELIMINATING DATA REDUNDANCY**

In [ ]:
duplicates = df_clean_noise[df_clean_noise.duplicated(keep=False)]
print("Duplicate Rows found:")
print(duplicates)

# Remove duplicates (keep the first occurrence)
df_final = df_clean_noise.drop_duplicates(keep='first')

print("\n--- FINAL CLEANED DATAFRAME ---")
print(df_final)

Duplicate Rows found:
     Name   Age  Salary      City
0  Akshay  25.0   50000  Nizambad
2  Akshay  25.0   50000  Nizambad

--- FINAL CLEANED DATAFRAME ---
          Name   Age   Salary          City
0       Akshay  25.0    50000      Nizambad
1   Mahesh Bob  32.5    60000       Lucknow
3  Kalyan babu  45.0    80000  Chilka nagar
5      PraBoss  30.0    58000    Pittapuram
6  Young Tiger  32.5    62000      Nizambad
7       Cherry  35.0  2000000  Saroor nagar


Lab 1 practice


In [ ]:
from google.colab import files
uploaded=files.upload()

Saving Employee_data.xlsx to Employee_data.xlsx


In [ ]:
import pandas as pd

# Read Excel file
df1 = pd.read_excel("Employee_data.xlsx")

# Count missing values per column
missing_per_column = df1.isnull().sum()

print("Missing values per column:")
print(missing_per_column)

# Total missing values in the entire file
total_missing = df1.isnull().sum().sum()
print("\nTotal missing values:", total_missing)

Missing values per column:
Employee_ID    0
Name           0
Department     0
Age            2
Salary         2
Join_Date      0
dtype: int64

Total missing values: 4


In [ ]:
# Method 1: Drop rows with missing values (Destructive)
df1_dropped = df1.dropna()
print("1. Shape after dropping rows with NaNs:", df1_dropped.shape)

print(df1_dropped)

1. Shape after dropping rows with NaNs: (46, 6)
    Employee_ID              Name   Department    Age    Salary  Join_Date
0           101          John Doe  Engineering   28.0   75000.0 2021-01-15
1           102        Jane Smith    Marketing   34.0   82000.0 2019-03-12
3           104    Sarah Williams           HR   29.0   62000.0 2022-02-20
4           105      Robert Brown        Sales   45.0   55000.0 2018-11-10
5           101          John Doe  Engineering   28.0   75000.0 2021-01-15
7           107      Chris Miller        Sales   22.0   48000.0 2023-01-05
8           108       Anna Taylor           HR  205.0   72000.0 2020-10-15
9           109      David Wilson  Engineering   40.0  120000.0 2015-05-19
10          110       Linda Moore        Sales   37.0   64000.0 2017-08-30
11          111    James Anderson          ???   29.0   59000.0 2022-04-12
12          112    Barbara Thomas    Marketing   33.0   78000.0 2019-09-01
13          113   Richard Jackson        Sales   42.

In [ ]:
# Method 2: Imputation (Filling with Mean/Median/Mode)
# We will use Median for Age to fill NaNs (robust to outliers)
df1_imputed = df1.copy()
median_age = df1_imputed['Age'].median()
df1_imputed['Age'] = df1_imputed['Age'].fillna(median_age)
median_salary = df1_imputed['Salary'].median()
df1_imputed['Salary'] = df1_imputed['Salary'].fillna(median_salary)

print(f"2. Filled missing Age and Salary with median ({median_age}),({median_salary}):")
print(df1_imputed)
print("\n")

2. Filled missing Age and Salary with median (32.0),(73500.0):
    Employee_ID              Name   Department    Age    Salary  Join_Date
0           101          John Doe  Engineering   28.0   75000.0 2021-01-15
1           102        Jane Smith    Marketing   34.0   82000.0 2019-03-12
2           103      Mike Johnson  Engineering   32.0   90000.0 2020-06-01
3           104    Sarah Williams           HR   29.0   62000.0 2022-02-20
4           105      Robert Brown        Sales   45.0   55000.0 2018-11-10
5           101          John Doe  Engineering   28.0   75000.0 2021-01-15
6           106       Emily Davis    Marketing   31.0   73500.0 2021-07-22
7           107      Chris Miller        Sales   22.0   48000.0 2023-01-05
8           108       Anna Taylor           HR  205.0   72000.0 2020-10-15
9           109      David Wilson  Engineering   40.0  120000.0 2015-05-19
10          110       Linda Moore        Sales   37.0   64000.0 2017-08-30
11          111    James Anderson    

In [ ]:
# We will use the IQR (Interquartile Range) method to remove Age outliers (e.g., 120).

Q1 = df1_imputed['Age'].quantile(0.25)
Q3 = df1_imputed['Age'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Age Bounds: {lower_bound} to {upper_bound}")

# Filter the data
df1_clean_noise = df1_imputed[
    (df1_imputed['Age'] >= lower_bound) &
    (df1_imputed['Age'] <= upper_bound)
]

print("Rows removed (Noise):")
print(df1_imputed[~df1_imputed.index.isin(df1_clean_noise.index)])
print("\n")
print("Dataset after noise removal")
print(df1_clean_noise)

Age Bounds: 15.875 to 50.875
Rows removed (Noise):
    Employee_ID            Name Department    Age   Salary  Join_Date
8           108     Anna Taylor         HR  205.0  72000.0 2020-10-15
20          119  Nancy Martinez      Sales   -5.0  63000.0 2018-04-10


Dataset after noise removal
    Employee_ID              Name   Department   Age    Salary  Join_Date
0           101          John Doe  Engineering  28.0   75000.0 2021-01-15
1           102        Jane Smith    Marketing  34.0   82000.0 2019-03-12
2           103      Mike Johnson  Engineering  32.0   90000.0 2020-06-01
3           104    Sarah Williams           HR  29.0   62000.0 2022-02-20
4           105      Robert Brown        Sales  45.0   55000.0 2018-11-10
5           101          John Doe  Engineering  28.0   75000.0 2021-01-15
6           106       Emily Davis    Marketing  31.0   73500.0 2021-07-22
7           107      Chris Miller        Sales  22.0   48000.0 2023-01-05
9           109      David Wilson  Engineer

In [ ]:
# We will use the IQR (Interquartile Range) method to remove Age outliers (e.g., 120).

Q1 = df1_imputed['Salary'].quantile(0.25)
Q3 = df1_imputed['Salary'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Salary Bounds: {lower_bound} to {upper_bound}")

# Filter the data
df1_clean_noise = df1_imputed[
    (df1_imputed['Salary'] >= lower_bound) &
    (df1_imputed['Salary'] <= upper_bound)
]

print("Rows removed (Noise):")
print(df1_imputed[~df1_imputed.index.isin(df1_clean_noise.index)])
print("\n")
print("Dataset after noise removal")
print(df1_clean_noise)



Salary Bounds: 30875.0 to 113875.0
Rows removed (Noise):
    Employee_ID             Name   Department   Age    Salary  Join_Date
9           109     David Wilson  Engineering  40.0  120000.0 2015-05-19
13          113  Richard Jackson        Sales  42.0  850000.0 2016-12-11
28          127      Jason Young  Engineering  44.0  115000.0 2014-07-18
42          140       Jack Perez  Engineering  29.0  999999.0 2021-03-30


Dataset after noise removal
    Employee_ID              Name   Department    Age    Salary  Join_Date
0           101          John Doe  Engineering   28.0   75000.0 2021-01-15
1           102        Jane Smith    Marketing   34.0   82000.0 2019-03-12
2           103      Mike Johnson  Engineering   32.0   90000.0 2020-06-01
3           104    Sarah Williams           HR   29.0   62000.0 2022-02-20
4           105      Robert Brown        Sales   45.0   55000.0 2018-11-10
5           101          John Doe  Engineering   28.0   75000.0 2021-01-15
6           106       E

In [ ]:
duplicates = df1_clean_noise[df1_clean_noise.duplicated(keep=False)]
print("Duplicate Rows found:")
print(duplicates)

# Remove duplicates (keep the first occurrence)
df1_final = df1_clean_noise.drop_duplicates(keep='first')

print("\n--- FINAL CLEANED DATAFRAME ---")
print(df1_final)

Duplicate Rows found:
    Employee_ID            Name   Department   Age   Salary  Join_Date
0           101        John Doe  Engineering  28.0  75000.0 2021-01-15
5           101        John Doe  Engineering  28.0  75000.0 2021-01-15
12          112  Barbara Thomas    Marketing  33.0  78000.0 2019-09-01
16          112  Barbara Thomas    Marketing  33.0  78000.0 2019-09-01
29          128      Alice King    Marketing  31.0  80000.0 2019-05-05
30          128      Alice King    Marketing  31.0  80000.0 2019-05-05

--- FINAL CLEANED DATAFRAME ---
    Employee_ID              Name   Department    Age    Salary  Join_Date
0           101          John Doe  Engineering   28.0   75000.0 2021-01-15
1           102        Jane Smith    Marketing   34.0   82000.0 2019-03-12
2           103      Mike Johnson  Engineering   32.0   90000.0 2020-06-01
3           104    Sarah Williams           HR   29.0   62000.0 2022-02-20
4           105      Robert Brown        Sales   45.0   55000.0 2018-11-1